In [12]:
import requests
import json
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
spotify_client_id = '876ffbaf268446ad9dcff8d3aa7d303c'
spotify_client_secret = '8905006200144cd699bd4ad98fd9ace9'
client_credentials_manager = SpotifyClientCredentials(client_id=spotify_client_id, client_secret=spotify_client_secret)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

last_fm_api_key = '9972df8a61bcb357f45cbe46addb4d9a'
# artists = ['Rema']


In [14]:

url = f'http://ws.audioscrobbler.com/2.0/?method=chart.gettopartists&api_key={last_fm_api_key}&format=json&limit=1000'
response = requests.get(url)
top_artists = response.json()['artists']['artist']
artists = [artist['name'] for artist in top_artists]
artists = artists[:500]

#artists = ['Radiohead', 'Coldplay']
def get_top_tracks(artist, api_key):
    url = f'http://ws.audioscrobbler.com/2.0/?method=artist.gettoptracks&artist={artist}&api_key={api_key}&format=json'
    response = requests.get(url)
    data = json.loads(response.text)
    top_tracks = data['toptracks']['track']
    tracks = []
    for track in top_tracks:
        track_dict = {}
        track_dict['name'] = track['name']
        track_dict['playcount'] = track['playcount']
        track_dict['listeners'] = track['listeners']
        track_dict['artist'] = track['artist']['name']
        tracks.append(track_dict)
    return tracks

def get_data(artists):
    top_tracks = []
    for artist in artists:
        tracks = get_top_tracks(artist, last_fm_api_key)
        top_tracks += tracks

    def get_audio_features(track_id, spotify):
        try:
            features = spotify.audio_features(track_id)[0]
        except requests.exceptions.ReadTimeout:
            # Wait for 1 second and try again
            time.sleep(1)
            features = get_audio_features(track_id, spotify)
#         features = spotify.audio_features(track_id)[0]
        if features is not None:
            features_dict = {}
            features_dict['danceability'] = features['danceability']
            features_dict['energy'] = features['energy']
            features_dict['key'] = features['key']
            features_dict['loudness'] = features['loudness']
            features_dict['mode'] = features['mode']
            features_dict['speechiness'] = features['speechiness']
            features_dict['acousticness'] = features['acousticness']
            features_dict['instrumentalness'] = features['instrumentalness']
            features_dict['liveness'] = features['liveness']
            features_dict['valence'] = features['valence']
            features_dict['tempo'] = features['tempo']
            features_dict['duration_ms'] = features['duration_ms']
            features_dict['time_signature'] = features['time_signature']
            return features_dict
        else:
            return []


    for track in top_tracks:
        results = spotify.search(q=f"{track['name']} {track['artist']}", type='track', limit=1)
        if results['tracks']['items']:
            track_id = results['tracks']['items'][0]['id']
            features = get_audio_features(track_id, spotify)
            track.update(features)


    # Assume that the variables 'artists' and 'top_tracks' are already defined
    data = []
    for artist in artists:
        artist_tracks = [track for track in top_tracks if track['artist'] == artist]
        for key in ['acousticness','album_name','popularity', 'danceability', 'energy', 'instrumentalness', 'key',
                'loudness', 'mode', 'speechiness', 'liveness', 'valence', 'tempo',
                'duration_ms', 'time_signature','track_genre']:
            if key not in track:
                track[key] = float('nan')
        for track in artist_tracks:
            print(track)
            track_data = {
                'artist': artist,
                'album_name': track.get('album_name', float('nan')),
                'track_name': track['name'],
                'popularity': track.get('popularity', float('nan')),
                'acousticness': track.get('acousticness', float('nan')),
                'danceability': track.get('danceability', float('nan')),
                'energy': track.get('energy', float('nan')),
                'instrumentalness': track.get('instrumentalness', float('nan')),
                'key': track.get('key', float('nan')),
                'loudness': track.get('loudness', float('nan')),
                'mode': track.get('mode', float('nan')),
                'speechiness': track.get('speechiness', float('nan')),
                'liveness': track.get('liveness', float('nan')),
                'valence': track.get('valence', float('nan')),
                'tempo': track.get('tempo', float('nan')),
                'duration_ms': track.get('duration_ms', float('nan')),
                'time_signature': track.get('time_signature', float('nan')),
                'track_genre': track.get('track_genre', float('nan'))
            }
            data.append(track_data)
    return(data)


df = pd.DataFrame(get_data(artists))
# df.to_csv('dataset.csv', index=False)

{'name': 'Calm Down (with Selena Gomez)', 'playcount': '2509652', 'listeners': '283657', 'artist': 'Rema', 'danceability': 0.801, 'energy': 0.806, 'key': 11, 'loudness': -5.206, 'mode': 1, 'speechiness': 0.0381, 'acousticness': 0.382, 'instrumentalness': 0.000669, 'liveness': 0.114, 'valence': 0.802, 'tempo': 106.999, 'duration_ms': 239318, 'time_signature': 4}
{'name': 'Calm Down', 'playcount': '877788', 'listeners': '110965', 'artist': 'Rema', 'danceability': 0.801, 'energy': 0.806, 'key': 11, 'loudness': -5.206, 'mode': 1, 'speechiness': 0.0381, 'acousticness': 0.382, 'instrumentalness': 0.000669, 'liveness': 0.114, 'valence': 0.802, 'tempo': 106.999, 'duration_ms': 239318, 'time_signature': 4}
{'name': 'Dumebi', 'playcount': '458241', 'listeners': '55493', 'artist': 'Rema', 'danceability': 0.923, 'energy': 0.657, 'key': 3, 'loudness': -5.173, 'mode': 0, 'speechiness': 0.0605, 'acousticness': 0.199, 'instrumentalness': 0.00407, 'liveness': 0.108, 'valence': 0.734, 'tempo': 110.015, 